<a href="https://colab.research.google.com/github/anindabitm/InstaDeep-Enzyme-Classification-Challenge/blob/main/CNN_NB2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
import os
import gc
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pickle
from sklearn.model_selection import train_test_split,StratifiedKFold
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.regularizers import l2
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical,plot_model
from keras.layers import Dense, Input, GlobalMaxPooling1D,Layer,GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM,Input,Bidirectional,Dropout,Add,BatchNormalization,Activation,Flatten,Attention,Concatenate
from keras.models import Model, Sequential
from keras.initializers import Constant
from keras import metrics,layers

In [ ]:
%tensorflow_version 2.x
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.4.1
Running on TPU  ['10.103.97.2:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.103.97.2:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.103.97.2:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
import os
os.chdir('/content/drive/My Drive/instadeep')

# Reading train data

In [ ]:
train = pd.read_csv('Train1.csv')
train.shape

(858777, 5)

In [ ]:
with open('keras-tokenizer/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train.SEQUENCE.values) #Converting text to a vector of word indexes

Found 24 unique tokens.


In [ ]:
le = LabelEncoder()
train.LABEL = le.fit_transform(train.LABEL.values)

In [ ]:
trainvalid_data2 = pad_sequences(train_sequences, padding='post',truncating='post',maxlen=MAX_SEQUENCE_LENGTH)
np.save('trainvalid2',trainvalid_data2)

In [ ]:
trainvalid_data1 = np.load('trainvalid2.npy')
trainvalid_data2 = np.fliplr(trainvalid_data1)
trainvalid_labels = train.LABEL.values

In [ ]:
trainvalid_data1.shape,trainvalid_data2.shape

((858777, 512), (858777, 512))

In [ ]:
def simplemodel():
  input1 = Input(shape=(512,))
  input2 = Input(shape=(512,))
  emb1 = Embedding(25, 50,input_length=512)(input1)
  emb2 = Embedding(25, 50,input_length=512)(input2)
  conv1 = Conv1D(4096, 35, activation='relu')(emb1)
  drop1 = Dropout(0.30)(conv1)
  pool11 = GlobalMaxPooling1D()(drop1)
  pool12 = GlobalAveragePooling1D()(drop1)
  conv2 = Conv1D(4096, 35, activation='relu')(emb2)
  drop2 = Dropout(0.30)(conv2)
  pool21 = GlobalMaxPooling1D()(drop2)
  pool22 = GlobalAveragePooling1D()(drop2)
  x = Concatenate()([pool11,pool12,pool21,pool22])
  x = Dense(256, activation='relu')(x)
  x = Dropout(0.30)(x)
  x = Dense(128, activation='relu')(x)
  x = Dropout(0.30)(x)
  output = Dense(20, activation='softmax')(x)
  model = Model(inputs=[input1,input2],outputs=output)
  model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=[keras.metrics.SparseCategoricalAccuracy()])
  
  return model

In [ ]:
def get_model():
  with tpu_strategy.scope():
    # creating the model in the TPUStrategy scope means we will train the model on the TPU
    model = simplemodel()
    
  return model

# Train and Prediction

In [ ]:
test = pd.read_csv('Test.csv')
print(test.shape)
test.head()

(253146, 3)


,SEQUENCE_ID,SEQUENCE,CREATURE
0,000LRRAI,MKLPVKRYAVAAIVALGVSMAPGELRTSPEAQIKIATREECRATPY...,creature7
1,000O8L4W,MKISVFGSGYVGLVQAAVLAEVGHDVVCMDIDKVKVEQLSQGQVHI...,creature6
2,000UKB5J,MTMKVGFIGLGIMGKPMSKNLLKAGYSLVVADRNPEAIADVIAAGA...,creature7
3,00111SLH,MTVSLSRLERQLGYTFKDQELMILALTHRSFAGRNNERLEFLGDAI...,creature6
4,001ELB0A,MSKIRVLSVDDSALMRQIMTEIINSHSDMEMVATAPDPLVARDLIK...,creature7


In [ ]:
ss = pd.read_csv('SampleSubmission.csv')
ss.head()

,SEQUENCE_ID,LABEL
0,000LRRAI,NaN
1,000O8L4W,NaN
2,000UKB5J,NaN
3,00111SLH,NaN
4,001ELB0A,NaN


In [ ]:
test_sequences = tokenizer.texts_to_sequences(test.SEQUENCE.values) #Converting text to a vector of word indexes

In [ ]:
test_data1 = pad_sequences(test_sequences, maxlen=512,padding='post',truncating='post')
test_data2 = np.fliplr(test_data1)

In [ ]:
def run_fold(fold):
  preds = np.zeros(shape=(test.shape[0],20))
  checkpoint = tf.keras.callbacks.ModelCheckpoint(f'/content/drive/My Drive/instadeep/keras-model/simplemodel_fold_post_{fold}.h5', monitor='val_loss', mode='min', save_best_only=True)
  train_index = train[train['fold'] == fold].index
  test_index =  train[train['fold'] != fold].index
  x_train1 = trainvalid_data1[train_index]
  x_train2 = trainvalid_data2[train_index]
  x_val1 = trainvalid_data1[test_index]
  x_val2 = trainvalid_data2[test_index]
  y_train = trainvalid_labels[train_index]
  y_val = trainvalid_labels[test_index]
  print(x_train1.shape,x_val1.shape,y_train.shape,y_val.shape)
  train_dataset = tf.data.Dataset.from_tensor_slices(((x_train1,x_train2),y_train)).shuffle(2048).batch(batch_size).prefetch(AUTO)
  val_dataset = tf.data.Dataset.from_tensor_slices(((x_val1,x_val2),y_val)).batch(batch_size).prefetch(AUTO)
  model = get_model()
  history = model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCH,callbacks=checkpoint)
  preds += model.predict([test_data1,test_data2])
  final_preds = pd.DataFrame(data=preds,columns=le.classes_)
  final_preds.to_csv(f'preds_fold_post{fold}.csv',index=False)
  del train_dataset,val_dataset,model
  gc.collect()


In [ ]:
EPOCH = 7
AUTO = tf.data.experimental.AUTOTUNE
batch_size = 8 * tpu_strategy.num_replicas_in_sync
run_fold(1)
run_fold(2)
run_fold(3)
run_fold(4)
run_fold(5)

(171756, 512) (687021, 512) (171756,) (687021,)
Epoch 1/7
2684/2684 [==============================] - 431s 156ms/step - loss: 1.3396 - sparse_categorical_accuracy: 0.6030 - val_loss: 0.1620 - val_sparse_categorical_accuracy: 0.9600
Epoch 2/7
2684/2684 [==============================] - 402s 150ms/step - loss: 0.1777 - sparse_categorical_accuracy: 0.9534 - val_loss: 0.1103 - val_sparse_categorical_accuracy: 0.9711
Epoch 3/7
2684/2684 [==============================] - 403s 150ms/step - loss: 0.1138 - sparse_categorical_accuracy: 0.9694 - val_loss: 0.0879 - val_sparse_categorical_accuracy: 0.9787
Epoch 4/7
2684/2684 [==============================] - 402s 150ms/step - loss: 0.0931 - sparse_categorical_accuracy: 0.9751 - val_loss: 0.0911 - val_sparse_categorical_accuracy: 0.9778
Epoch 5/7
2684/2684 [==============================] - 400s 149ms/step - loss: 0.0757 - sparse_categorical_accuracy: 0.9801 - val_loss: 0.0910 - val_sparse_categorical_accuracy: 0.9778
Epoch 6/7
2684/2684 [======